In [1]:
import os
import sys
import comtypes.client

In [2]:
# False,表示脚本将启动一个新的sap2000实例，而不是连接已有的。-- False方案好像已失效，以后自动手动打开软件后，用True方案。
# AttachToInstance = False
AttachToInstance = True
# True则可以手动指定sap2000的安装路径，False则不会执行安装路径的代码，而会自动打开最新安装的sap2000版本。
SpecifyPath = True
ProgramPath = r'C:\Program Files\Computers and Structures\SAP2000 22\SAP2000.exe'
# 设置模型文件的保存路径。
APIPath = r'C:\Users\Administrator\Desktop\SAP'
if not os.path.exists(APIPath): # 如果不存在，则创建
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
# 拼接，os.sep: 操作系统的分隔符（在windows下是 \
ModelPath = APIPath + os.sep + 'API_6-001.sdb'
if AttachToInstance:
    #attach to a running instance of SAP2000
    # 连接到已打开SAP2000实例
    try:
        #get the active SapObject
        # 尝试获取当前正在运行的SAP2000实例的对象，"CSI.SAP2000.API.SapObject"是SAP2000在系统注册表中的COM接口名。
        mySapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    #create API helper object
    # 创建一个新的实例
    # 'SAP2000v22.Helper' SAP2000提供的COM辅助类，用来启动主程序、创建对象等。
    helper = comtypes.client.CreateObject('SAP2000v22.Helper')
    # 调用QueryInterface将对象转换为具体的cHelper接口，这样才能使用其中定义的方法。
    helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
    if SpecifyPath:
        try:
            #'create an instance of the SAPObject from the specified path
            mySapObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try:
            #create an instance of the SAPObject from the latest installed SAP2000
            mySapObject = helper.CreateObjectProgID("CSI.SAP2000.API.SapObject")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
        #start SAP2000 application
        mySapObject.ApplicationStart()


In [3]:
#create SapModel object
# 从SAP2000主对象中提取出SapModel，这是所有结构建模操作的核心对象，之后定义节点、杆件、荷载等操作都通过它来进行。
SapModel = mySapObject.SapModel

In [26]:
#initialize model
SapModel.InitializeNewModel()

0

In [27]:
#create new blank model
ret = SapModel.File.NewBlank()

In [28]:
#switch to k-ft units
kip_ft_F = 4
ret = SapModel.SetPresentUnits(kip_ft_F)

In [14]:
#add frame object by coordinates
LinkName1 = ' '
# 创建link，False是连接两单，否则单点
[LinkName1, ret] = SapModel.LinkObj.AddByCoord(0, 0, 0,0,0,0, LinkName1,True, 'R1', '1', 'Global')

In [71]:
# link属性 damper类型
MyDOF = [0]*6
MyFixed = [0]*6
MyNonLinear = [0]*6
MyKe = [0]*6 # 链接属性的有效刚度项的数组，有效刚度适用于线性分析
MyCe = [0]*6
MyK = [0]*6 # 链接属性的初始刚度项数组，初始刚度适用于非线性分析
MyC = [0]*6 # 非线性阻尼
MyCexp = [0]*6

MyDOF[0] = True
MyNonLinear[0] = True
# 非线性分析
MyK[0] = 10000
MyC[0]= 1
MyCexp[0]= 1

ret = SapModel.PropLink.SetDamper("D1", MyDOF, MyFixed, MyNonLinear, MyKe, MyCe, MyK, MyC, MyCexp, 1, 0)

ret = SapModel.PropLink.SetSpringData("D1", 1, 1)
ret

0

In [73]:
# Link属性linear类型 DOF 自由方向矩阵、Fixed 固定、Ke刚度、Ce阻尼。
DOF = [0]*6
Fixed = [0]*6
Ke = [0]*6
Ce = [0]*6
DOF[0] = True
Ke[0] = 100
ret = SapModel.PropLink.SetLinear("L1", DOF, Fixed, Ke, Ce, 0, 0)
ret
ret = SapModel.PropLink.SetSpringData("L1", 1, 1)
ret

0

In [93]:
#add frame object by coordinates
LinkName1 = ' '
# 创建link，False是连接两单，否则单点并忽略第二个坐标,并设置连接属性，
[LinkName1, ret] = SapModel.LinkObj.AddByCoord(0, 0, 0, 0, 0, 10, LinkName1,True,PropName="L1")

In [94]:
#add frame object by coordinates
LinkName2 = ' '
# 创建link，False是连接两点连接，否则单点
[LinkName2, ret] = SapModel.LinkObj.AddByCoord(0, 0, 0, 0, 0, 10, LinkName2,True,PropName="D1")
ret


0

In [89]:
# 定义时程函数
ret = SapModel.Func.FuncTH.SetSine("SINE", 0.6283, 100, 41, 1)


In [90]:
# 添加一个工况模式，
ret = SapModel.LoadPatterns.Add("1",8)


In [108]:
# 给节点添加荷载
value = [0]*6
value[2]=10
ret = SapModel.PointObj.SetLoadForce("1", "1", value)
ret

[(0.0, 0.0, 10.0, 0.0, 0.0, 0.0), 0]

In [109]:
# 给节点添加质量
mass = [0]*6
mass[2] = 1 # u3方向加质量
ret = SapModel.PointObj.SetMass("1",mass) # True 覆盖 False 叠加
ret

[(0.0, 0.0, 1.0, 0.0, 0.0, 0.0), 0]

In [110]:
# 模态工况
ret = SapModel.LoadCases.ModalEigen.SetCase("MODAL")
ret

0

In [114]:
ret = SapModel.LoadCases.ModHistNonlinear.SetCase("NLMHIST1")
# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['1'] # 荷载模式名
MyFunc = ['SINE']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.ModHistNonlinear.SetLoads("NLMHIST1", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.ModHistNonlinear.SetTimeStep("NLMHIST1", 2550, 0.01)
# 阻尼
ret = SapModel.LoadCases.ModHistNonlinear.SetDampConstant("NLMHIST1", 0)

In [116]:
ret = SapModel.LoadCases.DirHistNonlinear.SetCase("NLDHIST1")
# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['1'] # 荷载模式名
MyFunc = ['SINE']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.DirHistNonlinear.SetLoads("NLDHIST1", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.DirHistNonlinear.SetTimeStep("NLDHIST1", 2550, 0.01)

